In [44]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures


from sklearn.naive_bayes import GaussianNB
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.metrics import confusion_matrix, classification_report
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
import random
from sklearn.model_selection import KFold
from datetime import datetime

In [45]:
X = pd.read_csv('../datasets/speaker_formula_b.csv')

In [46]:
target_column = 'language'
y = X[target_column]
X_ = X.drop([target_column], axis=1)
features = list(X_)
print(features)

['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10', 'X11', 'X12', 'cos(2*pi*(X1-min(X1)+X2-min(X2)))/(max(X1)+max(X2)-min(X1)-min(X2))', 'cos(2*pi*(X1-min(X1)+X3-min(X3)))/(max(X1)+max(X3)-min(X1)-min(X3))', 'cos(2*pi*(X1-min(X1)+X4-min(X4)))/(max(X1)+max(X4)-min(X1)-min(X4))', 'cos(2*pi*(X1-min(X1)+X5-min(X5)))/(max(X1)+max(X5)-min(X1)-min(X5))', 'cos(2*pi*(X1-min(X1)+X6-min(X6)))/(max(X1)+max(X6)-min(X1)-min(X6))', 'cos(2*pi*(X1-min(X1)+X7-min(X7)))/(max(X1)+max(X7)-min(X1)-min(X7))', 'cos(2*pi*(X1-min(X1)+X8-min(X8)))/(max(X1)+max(X8)-min(X1)-min(X8))', 'cos(2*pi*(X1-min(X1)+X9-min(X9)))/(max(X1)+max(X9)-min(X1)-min(X9))', 'cos(2*pi*(X1-min(X1)+X10-min(X10)))/(max(X1)+max(X10)-min(X1)-min(X10))', 'cos(2*pi*(X1-min(X1)+X11-min(X11)))/(max(X1)+max(X11)-min(X1)-min(X11))', 'cos(2*pi*(X1-min(X1)+X12-min(X12)))/(max(X1)+max(X12)-min(X1)-min(X12))', 'cos(2*pi*(X2-min(X2)+X3-min(X3)))/(max(X2)+max(X3)-min(X2)-min(X3))', 'cos(2*pi*(X2-min(X2)+X4-min(X4)))/(max(X2)+max(X4)-min(X2)-

### KNN

In [37]:
%%time
print(f"Now => {datetime.now()}")
kf = KFold(n_splits=5, shuffle=True, random_state=random.randint(100, 2000))
i = 1
results = []
for train_index, test_index in kf.split(X):
    X_train = X.iloc[train_index].loc[:, features]
    X_test = X.iloc[test_index][features]
    y_train = X.iloc[train_index].loc[:, target_column]
    y_test = X.loc[test_index][target_column]
    
    plain_sfs = SFS(KNeighborsClassifier(), 
          k_features=(5, 10), 
          forward=False, 
          floating=False,
          n_jobs=-1)
    
    plain_sfs.fit(X_train, y_train)
    selected_features = X.columns[list(plain_sfs.k_feature_idx_)]
    print(f'Features => {list(selected_features)}')
    x_t = X_test[selected_features]
    
    clf = KNeighborsClassifier()
    clf.fit(X_train[selected_features], y_train)
    score = accuracy_score(y_test, clf.predict(x_t))
    results.append(score)
    print(f"Accuracy for the fold no. {i} on the test set: {score}\n")
    i += 1
    
print(f'KNN {np.array(results).mean()}')

Now => 2022-02-20 11:51:01.919833
Features => ['RI', 'Na', 'Mg', 'Na*Mg', 'Na*Al', 'Na*Si', 'Na*Ca', 'Mg*Si', 'Si*Ca', 'Si*Fe']
Accuracy for the fold no. 1 on the test set: 0.5348837209302325

Features => ['RI', 'Na', 'Mg', 'Na*Mg', 'Na*Ca', 'Mg*Ca', 'Al*Si', 'Al*Ca', 'Si*K', 'Si*Ba']
Accuracy for the fold no. 2 on the test set: 0.813953488372093

Features => ['Ca', 'Na*Si', 'Na*Ca', 'Al*Si', 'Al*Ca']
Accuracy for the fold no. 3 on the test set: 0.5116279069767442

Features => ['RI', 'Na', 'Mg', 'Al', 'RI*Na', 'Na*Mg', 'Na*Ca', 'Al*Si', 'Si*K', 'Si*Ba']
Accuracy for the fold no. 4 on the test set: 0.6511627906976745

Features => ['RI', 'Na', 'Mg', 'Al', 'Na*Si', 'Mg*Si', 'Mg*Ca', 'Al*Si', 'Si*K', 'Si*Ba']
Accuracy for the fold no. 5 on the test set: 0.5238095238095238

KNN 0.6070874861572536
CPU times: user 11.8 s, sys: 1.12 s, total: 12.9 s
Wall time: 22.2 s


#### Logistic Regression

In [38]:
%%time
print(f"Now => {datetime.now()}")
kf = KFold(n_splits=5, shuffle=True, random_state=321)
i = 1
results = []
for train_index, test_index in kf.split(X):
    X_train = X.iloc[train_index].loc[:, features]
    X_test = X.iloc[test_index][features]
    y_train = X.iloc[train_index].loc[:, target_column]
    y_test = X.loc[test_index][target_column]
    
    plain_sfs_ls = SFS(LogisticRegression(), 
          k_features=(5, 10), 
          forward=False, 
          floating=False,
          n_jobs=-1)
    
    plain_sfs_ls.fit(X_train, y_train)
    selected_features = X.columns[list(plain_sfs_ls.k_feature_idx_)]
    print(f'Features => {list(selected_features)}')
    x_t = X_test[selected_features]
    
    clf = LogisticRegression()
    clf.fit(X_train[selected_features], y_train)
    score = accuracy_score(y_test, clf.predict(x_t))
    results.append(score)
    print(f"Accuracy for the fold no. {i} on the test set: {score}\n")
    i += 1
    
print(f'LogisticRegression {np.array(results).mean()}')

Now => 2022-02-20 11:51:24.167225
Features => ['Ba', 'RI*Fe', 'Na*Mg', 'Na*Si', 'Al*Si', 'Al*Ca', 'Si*K', 'Si*Ca', 'Si*Ba', 'Si*Fe']
Accuracy for the fold no. 1 on the test set: 0.5348837209302325

Features => ['RI*Al', 'Na*Si', 'Mg*Al', 'Mg*Ca', 'Mg*Ba', 'Al*K', 'Al*Ba', 'Si*Ca', 'Si*Ba']
Accuracy for the fold no. 2 on the test set: 0.6511627906976745

Features => ['Si', 'RI*Fe', 'Na*Si', 'Na*Ca', 'Mg*Ca', 'Al*Si', 'Al*Ca', 'Al*Ba', 'Si*K', 'Si*Ba']
Accuracy for the fold no. 3 on the test set: 0.5581395348837209

Features => ['Na', 'Na*Mg', 'Na*K', 'Na*Ba', 'Al*Si', 'Al*Ca', 'Si*K', 'Si*Ca', 'Si*Fe']
Accuracy for the fold no. 4 on the test set: 0.6976744186046512

Features => ['Al', 'Si', 'Na*Mg', 'Mg*Ca', 'Al*Si', 'Al*Ca', 'Si*K', 'Si*Ca', 'Si*Fe']
Accuracy for the fold no. 5 on the test set: 0.5952380952380952

LogisticRegression 0.6074197120708749
CPU times: user 12.7 s, sys: 1.02 s, total: 13.7 s
Wall time: 3min 37s


### Naive Bayes

In [39]:
%%time
print(f"Now => {datetime.now()}")
i = 1
results = []
kf = KFold(n_splits=5, shuffle=True, random_state=3421)
for train_index, test_index in kf.split(X):
    X_train = X.iloc[train_index].loc[:, features]
    X_test = X.iloc[test_index][features]
    y_train = X.iloc[train_index].loc[:, target_column]
    y_test = X.loc[test_index][target_column]
    
    naive_bayes = GaussianNB()
    plain_sfs_nb = SFS(naive_bayes, 
          k_features=(5, 10), 
          forward=False, 
          floating=False)
    
    plain_sfs_nb.fit(X_train, y_train)
    selected_features = X.columns[list(plain_sfs_nb.k_feature_idx_)]
    print(f'Features => {list(selected_features)}')
    x_t = X_test[selected_features]
    
    clf = GaussianNB()
    clf.fit(X_train[selected_features], y_train)
    score = accuracy_score(y_test, clf.predict(x_t))
    results.append(score)
    print(f"Accuracy for the fold no. {i} on the test set: {score}\n")
    i += 1
    
print(f'Naive Bayes {np.array(results).mean()}')

Now => 2022-02-20 11:55:01.373973
Features => ['Al', 'K', 'Ca', 'Ba', 'Fe', 'RI*Al', 'Na*Si', 'Na*K', 'Al*K', 'Si*K']
Accuracy for the fold no. 1 on the test set: 0.627906976744186

Features => ['Na', 'Mg', 'Al', 'K', 'RI*Mg', 'RI*Al', 'RI*Si', 'Na*Al', 'Si*Ca', 'K*Fe']
Accuracy for the fold no. 2 on the test set: 0.3953488372093023

Features => ['Al', 'RI*Ba', 'Si*K', 'Si*Ca', 'Ba*Fe']
Accuracy for the fold no. 3 on the test set: 0.5348837209302325

Features => ['Al', 'K', 'RI*Al', 'RI*Ca', 'Na*Al', 'Na*K']
Accuracy for the fold no. 4 on the test set: 0.5348837209302325

Features => ['Al', 'K', 'Ca', 'Ba', 'RI*K', 'Si*Ca']
Accuracy for the fold no. 5 on the test set: 0.40476190476190477

Naive Bayes 0.49955703211517166
CPU times: user 47.1 s, sys: 498 ms, total: 47.6 s
Wall time: 48.2 s


### SVM

In [40]:
%%time
print(f"Now => {datetime.now()}")

kf = KFold(n_splits=5, shuffle=True, random_state=random.randint(1000,9999))
i = 1
results = []
for train_index, test_index in kf.split(X):
    X_train = X.iloc[train_index].loc[:, features]
    X_test = X.iloc[test_index][features]
    y_train = X.iloc[train_index].loc[:, target_column]
    y_test = X.loc[test_index][target_column]
    
    plain_sfs_svm = SFS(svm.SVC(kernel='linear'), 
          k_features=(2, 10), 
          forward=False, 
          floating=False,
          n_jobs=-1)
    
    plain_sfs_svm.fit(X_train, y_train)
    selected_features = X.columns[list(plain_sfs_svm.k_feature_idx_)]
    print(f'Features => {list(selected_features)}')
    x_t = X_test[selected_features]
    
    clf = svm.SVC(kernel='linear')
    clf.fit(X_train[selected_features], y_train)
    score = accuracy_score(y_test, clf.predict(x_t))
    results.append(score)
    print(f"Accuracy for the fold no. {i} on the test set: {score}\n")
    i += 1
    
print(f'SVM {np.array(results).mean()}')

Now => 2022-02-20 11:55:49.610253
Features => ['Al', 'K', 'Na*Mg', 'Na*Al', 'Na*Si', 'Mg*Ca', 'Si*Ca', 'K*Ca']
Accuracy for the fold no. 1 on the test set: 0.5581395348837209

Features => ['RI*Si', 'Na*Mg', 'Na*Si', 'Na*K', 'Na*Ca', 'Al*Si', 'Si*Ca', 'Si*Ba', 'K*Ca']
Accuracy for the fold no. 2 on the test set: 0.6046511627906976

Features => ['Na', 'Si', 'Mg*Al', 'Mg*Si', 'Al*K', 'Si*Ca']
Accuracy for the fold no. 3 on the test set: 0.6976744186046512

Features => ['Mg', 'Si', 'K', 'RI*Ca', 'Na*Al', 'Na*Si', 'Mg*Si', 'Al*Si', 'Al*Ca']
Accuracy for the fold no. 4 on the test set: 0.6744186046511628

Features => ['RI*Si', 'Na*Si', 'Mg*Si', 'Mg*Ca', 'Al*Si', 'Al*Ca', 'Si*Ca', 'Si*Ba', 'Si*Fe', 'K*Ca']
Accuracy for the fold no. 5 on the test set: 0.5952380952380952

SVM 0.6260243632336655
CPU times: user 14.8 s, sys: 860 ms, total: 15.6 s
Wall time: 18min 25s


### Random Forest

In [41]:
%%time
print(f"Now => {datetime.now()}")

kf = KFold(n_splits=5, shuffle=True, random_state=random.randint(1000,9999))
i = 1
results = []
for train_index, test_index in kf.split(X):
    X_train = X.iloc[train_index].loc[:, features]
    X_test = X.iloc[test_index][features]
    y_train = X.iloc[train_index].loc[:, target_column]
    y_test = X.loc[test_index][target_column]
    
    rfc_sbs = SFS(RandomForestClassifier(random_state=0), 
          k_features=(3, 10), 
          forward=False, 
          floating=False, n_jobs=-1)

    
    rfc_sbs.fit(X_train, y_train)
    selected_features = X.columns[list(rfc_sbs.k_feature_idx_)]
    print(f'Features => {list(selected_features)}')
    x_t = X_test[selected_features]
    
    clf = RandomForestClassifier(random_state=0)
    clf.fit(X_train[selected_features], y_train)
    score = accuracy_score(y_test, clf.predict(x_t))
    results.append(score)
    print(f"Accuracy for the fold no. {i} on the test set: {score}\n")
    i += 1
    
print(f'RandomForest {np.array(results).mean()}')

Now => 2022-02-20 12:14:14.928897
Features => ['RI', 'RI*Al', 'Na*Mg', 'Al*Ca', 'Al*Ba', 'Si*K', 'Si*Ba', 'Ba*Fe']
Accuracy for the fold no. 1 on the test set: 0.7674418604651163

Features => ['RI', 'Mg', 'Fe', 'RI*Ba', 'Mg*Ca', 'Al*Ca', 'K*Ca', 'K*Ba']
Accuracy for the fold no. 2 on the test set: 0.7441860465116279

Features => ['RI', 'Mg', 'RI*Na', 'RI*Ba', 'Mg*Ca', 'Al*K', 'Al*Ca', 'Si*K', 'Ca*Fe']
Accuracy for the fold no. 3 on the test set: 0.7674418604651163

Features => ['RI', 'RI*Al', 'Na*Si', 'Mg*Si', 'Mg*Ba', 'Al*Ca', 'Al*Ba', 'Si*K', 'Si*Ba', 'Si*Fe']
Accuracy for the fold no. 4 on the test set: 0.7209302325581395

Features => ['RI', 'Na', 'Ba', 'RI*Mg', 'RI*Al', 'RI*Ca', 'Mg*Ca', 'Al*Fe', 'Si*K', 'Si*Ba']
Accuracy for the fold no. 5 on the test set: 0.7380952380952381

RandomForest 0.7476190476190476
CPU times: user 16.5 s, sys: 1.07 s, total: 17.6 s
Wall time: 12min 56s


### Multilayer Perceptron

In [ ]:
%%time
print(f"Now => {datetime.now()}")

kf = KFold(n_splits=5, shuffle=True, random_state=random.randint(1000,9999))
i = 1
results = []
for train_index, test_index in kf.split(X):
    X_train = X.iloc[train_index].loc[:, features]
    X_test = X.iloc[test_index][features]
    y_train = X.iloc[train_index].loc[:, target_column]
    y_test = X.loc[test_index][target_column]
    
    mlp_sbs = SFS(MLPClassifier(random_state=1), 
          k_features=(1, 10), 
          forward=False, 
          floating=False)
    
    mlp_sbs.fit(X_train, y_train)
    selected_features = X.columns[list(mlp_sbs.k_feature_idx_)]
    print(f'Features => {list(selected_features)}')
    x_t = X_test[selected_features]
    
    clf = MLPClassifier(random_state=1)
    clf.fit(X_train[selected_features], y_train)
    score = accuracy_score(y_test, clf.predict(x_t))
    results.append(score)
    print(f"Accuracy for the fold no. {i} on the test set: {score}\n")
    i += 1
    
print(f'MLP {np.array(results).mean()}')

Now => 2022-02-20 13:25:00.213558
Features => ['X1', 'X5', 'X8', 'X10', 'X11', 'cos(2*pi*(X1-min(X1)+X3-min(X3)))/(max(X1)+max(X3)-min(X1)-min(X3))']
Accuracy for the fold no. 1 on the test set: 0.696969696969697

